# Clickbait spoiling: Rerank

In [68]:
%cd /content

/content


In [69]:
import pandas as pd
    
def load_dataset(file_name):
    import json
    ret = []
    
    with open(file_name, encoding = "utf-8") as f:
        for i in f:
            i = eval(i)
            query = i['postText']
            spoiler = i['spoiler']
            paragraphs = i['targetParagraphs']
            
            if i['tags'][0] == 'multi':
                continue
                
            assert len(query) == 1
            query = query[0]

            if len(spoiler) != 1:
                spoiler = ' '.join(spoiler)
            else:
                spoiler = spoiler[0]

            ret += [{'query': query, 'uuid': i['uuid'], 'paragraphs': paragraphs, 'spoiler': spoiler}]
            
    return pd.DataFrame(ret)


#test_dataset = load_dataset('test.jsonl')
train_dataset = load_dataset('train.jsonl')
validation_dataset = load_dataset('validation.jsonl')
#test_pilot_200 = load_dataset('200_test.jsonl')
all_datasets = pd.concat([ train_dataset, validation_dataset])

# Reranking utilities

In [70]:
import os
os.environ['LANG'] = 'en_US.UTF-8'


In [71]:
%cd /content
!rm -rf pygaggle && pip uninstall -y pygaggle
!git clone  --recursive https://github.com/castorini/pygaggle.git # use master once that branch is merged
%cd pygaggle
!pip install --editable .


/content


NotImplementedError: ignored

In [ ]:
!pip3 install faiss-cpu

In [72]:
from pygaggle.rerank.base import Query, Text
import torch
from pygaggle.rerank.transformer import MonoT5, MonoBERT
from tqdm import tqdm

monoT5 = MonoT5()
monoBert = MonoBERT()

In [73]:
def rerank(datapoint, reranker):
    passages = datapoint['paragraphs']
    passages = zip(range(len(passages)), passages)
    documents = [Text(i[1], {'docid': i[0]}, 0) for i in passages]
    ret = sorted(reranker.rerank(Query(datapoint['query']), documents), key=lambda i: i.score, reverse=True)

    return [{'score': i.score, 'id': i.metadata['docid'], 'text': i.text} for i in ret]


In [74]:
def run_reranking(output_dir, model, dataset, ground_truth_file_name):
    import json
    
    with open( 'predictions.json', 'w') as preds, open( 'passage-truth.jsonl', 'w',encoding="utf-8") as passage_truth, open( 'phrase-truth.jsonl', 'w',encoding="utf-8") as phrase_truth, open(  ground_truth_file_name,encoding="utf-8") as f:
        for i in f:
            #print(i)
            i_parsed = eval(i)
            #print(i_parsed)
            if i_parsed['tags'][0].startswith('passage'):
                passage_truth.write(i.strip() + '\n')
            elif i_parsed['tags'][0].startswith('phrase'):
                phrase_truth.write(i.strip() + '\n')
        
        result = {}
        dataset = [i for _, i in dataset.iterrows()]
        for i in tqdm(dataset):
            result[i['uuid']] = rerank(i, model)

        preds.write(json.dumps(result))

In [80]:
run_reranking('all-passages-retrieval-t5/', monoBert, all_datasets, 'validation.jsonl')

100%|██████████| 3298/3298 [21:23<00:00,  2.57it/s]


In [88]:

filter = train_dataset["uuid"] == "0af11f6b-c889-4520-9372-66ba25cb7657"
train_dataset.where(filter)[:1]["spoiler"]

0    how about that morning we go throw?
Name: spoiler, dtype: object

In [83]:
import json
with open("predictions.json") as f:
  data=json.load(f)
out = data["0af11f6b-c889-4520-9372-66ba25cb7657"]
out.sort(key = lambda x: x['score'],reverse=True)
for i in out[:5]:
  print(i["score"],i["text"])

-0.009555554948747158 Welker was one of Brady’s favorite targets for six seasons from 2007 to 2012. It’s understandable him and Brady want to meet with both being in the same area. But Brady typically is all business during football season. Welker probably should have known what he was getting into when reaching out to his buddy.
-0.011282924562692642 Welker revealed Friday morning on a Miami radio station that he contacted Brady because he’ll be in town for Sunday’s game between the New England Patriots and Miami Dolphins at Gillette Stadium. It seemed like a perfect opportunity for the two to catch up.
-0.012663664296269417 "That’s the only thing we really have planned," Welker said of his upcoming workout with Brady. "It’s just funny. I’m sitting there trying to have dinner. ‘Hey, get your ass up here and let’s go throw.’ I’m like, ‘Aw jeez, man.’ He’s going to have me running like 2-minute drills in his backyard or something."
-0.01671535335481167 Maybe Brady will put a good word i

In [89]:
run_reranking('all-passages-retrieval-t5/', monoT5, all_datasets, 'validation.jsonl')

100%|██████████| 3298/3298 [07:43<00:00,  7.11it/s]


In [90]:

filter = train_dataset["uuid"] == "0af11f6b-c889-4520-9372-66ba25cb7657"
train_dataset.where(filter)[:1]["spoiler"]

0    how about that morning we go throw?
Name: spoiler, dtype: object

In [91]:
import json
with open("predictions.json") as f:
  data=json.load(f)
out = data["0af11f6b-c889-4520-9372-66ba25cb7657"]
out.sort(key = lambda x: x['score'],reverse=True)
for i in out[:5]:
  print(i["score"],i["text"])

-0.01008510123938322 "That’s the only thing we really have planned," Welker said of his upcoming workout with Brady. "It’s just funny. I’m sitting there trying to have dinner. ‘Hey, get your ass up here and let’s go throw.’ I’m like, ‘Aw jeez, man.’ He’s going to have me running like 2-minute drills in his backyard or something."
-0.016101330518722534 "I hit him up to do dinner Saturday night. He’s like, ‘I’m going to be flying in from Ann Arbor later (after the Michigan-Colorado football game), but how about that morning we go throw?’ " Welker said on WQAM, per The Boston Globe. "And I’m just sitting there, I’m like, ‘I was just thinking about dinner, but yeah, sure. I’ll get over there early and we can throw a little bit.’ "
-0.018095524981617928 Welker was one of Brady’s favorite targets for six seasons from 2007 to 2012. It’s understandable him and Brady want to meet with both being in the same area. But Brady typically is all business during football season. Welker probably should

# Sentence Retrieval

In [92]:
import nltk
nltk.download('punkt')

def load_dataset(file_name):
    from nltk import sent_tokenize
    import json
    ret = []
    
    with open(file_name, encoding="utf-8") as f:
        for i in f:
            i = eval(i)
            query = i['postText']
            spoiler = i['spoiler']
            paragraphs = i['targetParagraphs']
            
            if i['tags'][0] == 'multi':
                continue
                
            assert len(query) == 1
            query = query[0]
            
            if len(spoiler) != 1:
                spoiler = ' '.join(spoiler)
            else:
                spoiler = spoiler[0]
            
            sentences = []
            for paragraph in paragraphs:
                for sentence in sent_tokenize(paragraph):
                    sentences += [sentence]
            
            ret += [{'query': query, 'uuid': i['uuid'], 'paragraphs': sentences, 'spoiler': spoiler}]
            
    return ret


test_sent_dataset = load_dataset('validation.jsonl')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [93]:
def run_sent_reranking(output_dir, model, ground_truth_file_name):

    import json
    
    with open('predictions.json', 'w') as preds, open( 'passage-truth.jsonl', 'w',encoding="utf-8") as passage_truth, open('phrase-truth.jsonl', 'w',encoding="utf-8") as phrase_truth, open( ground_truth_file_name,encoding="utf-8") as f:
        for i in f:
            i_parsed = eval(i)
            if i_parsed['tags'][0].startswith('passage'):
                passage_truth.write(i.strip() + '\n')
            elif i_parsed['tags'][0].startswith('phrase'):
                phrase_truth.write(i.strip() + '\n')
    
        result = {}
        
        for i in tqdm(test_sent_dataset):
            result[i['uuid']] = rerank(i, model)
        
        preds.write(json.dumps(result))

In [94]:
run_sent_reranking('sentence-retrieval-t5/', monoT5, 'validation.jsonl')

100%|██████████| 657/657 [02:22<00:00,  4.61it/s]


In [106]:
import json
with open("predictions.json") as f:
  data=json.load(f)
out = data["a083b244-ea5d-43f9-a61b-26a0b7c5a70e"]
out.sort(key = lambda x: x['score'],reverse=True)
for i in out[:5]:
  print(i["score"],i["text"])

-0.047278422862291336 On Friday, the On Friday, the Houston Chronicle made its official endorsement known in an article titled: "These are unsettling times that require a steady hand: That’s not Donald Trump."
-0.050259966403245926 The Chronicle is very clear about how it came to its decision, despite The Chronicle is very clear about how it came to its decision, despite endorsing Mitt Romney in the last election.
-0.08166395872831345 The Houston Chronicle clearly recognizes the fact that the Republican presidential nominee has sent the Republican party into turmoil.
-0.22591888904571533 ‘The Chronicle editorial page does not typically endorse early in an election cycle; we prefer waiting for the campaign to play out and for issues to emerge and be addressed.
-0.30013835430145264 One of the largest newspapers in all of Texas, which One of the largest newspapers in all of Texas, which previously endorsed Republican presidential nominee Mitt Romney, has now officially endorsed Democratic

In [108]:
run_sent_reranking('test-200/sentence-retrieval-t5/', monoBert, 'validation.jsonl')

100%|██████████| 657/657 [09:11<00:00,  1.19it/s]


In [111]:
import json
with open("predictions.json") as f:
  data=json.load(f)
out = data["a083b244-ea5d-43f9-a61b-26a0b7c5a70e"]
out.sort(key = lambda x: x['score'],reverse=True)
for i in out[:5]:
  print(i["score"],i["text"])

-0.002375758718699217 On Friday, the On Friday, the Houston Chronicle made its official endorsement known in an article titled: "These are unsettling times that require a steady hand: That’s not Donald Trump."
-0.005145639646798372 The Houston Chronicle clearly recognizes the fact that the Republican presidential nominee has sent the Republican party into turmoil.
-0.015595500357449055 The Chronicle is very clear about how it came to its decision, despite The Chronicle is very clear about how it came to its decision, despite endorsing Mitt Romney in the last election.
-0.09569159895181656 ‘The Chronicle editorial page does not typically endorse early in an election cycle; we prefer waiting for the campaign to play out and for issues to emerge and be addressed.
-4.493777751922607 One of the largest newspapers in all of Texas, which One of the largest newspapers in all of Texas, which previously endorsed Republican presidential nominee Mitt Romney, has now officially endorsed Democratic 